In [ ]:
import pandas as pd
from COMETH import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time
import csv
gui = nimble.NimbleGUI()
gui.serve(8080)

# sudo lsof -iTCP:8080 -sTCP:LISTEN

In [ ]:
gui.nativeAPI().clear()
# Build skeleton
s12 = Skeleton('BODY12.xml')
# BSM
s = DynamicSkeleton(config='BODY15_constrained_3D.xml')
s.hip_correction = False
# gui.nativeAPI().renderSkeleton(s._nimble)
axis_length=0.2
gui.nativeAPI().createLine(key='x_axis',points=[np.array([0,0,0]),np.array([axis_length,0,0])],color=np.array([255,0,0,1]))
gui.nativeAPI().createLine(key='y_axis',points=[np.array([0,0,0]),np.array([0,axis_length,0])],color=np.array([0,255,0,1]))
gui.nativeAPI().createLine(key='z_axis',points=[np.array([0,0,0]),np.array([0,0,axis_length])],color=np.array([0,0,255,1]))

# Read data from CSV
# markers = pd.read_csv("~/nas/MAEVE/BEFINE/wearable_camera/TotalCapture/CommonFormat/vicon/s1/acting1/vicon_s1_acting1.csv")
# imus = pd.read_csv("~/nas/MAEVE/BEFINE/wearable_camera/TotalCapture/CommonFormat/imu/s1/acting1/imu_s1_acting1.csv")
markers = pd.read_csv("tmp/imu/vicon_s1_acting1.csv")
imus = pd.read_csv("tmp/imu/imu_s1_acting1.csv")

In [ ]:
# Build the markers dataframe with only the subset we are interested in

Rx = np.array([
    [1,0,0 ],
    [0,0,-1 ],
    [0,1,0]
])

markers.columns
markers_dict = {
'RKnee': 'right_knee',
'LWrist': 'left_wrist',
'RHip': 'right_hip',
'RShoulder': 'right_shoulder',
'LElbow': 'left_elbow',
'LHip': 'left_hip',
'RElbow': 'right_elbow',
'RWrist': 'right_wrist',
'LKnee': 'left_knee',
'LShoulder': 'left_shoulder',
'RAnkle': 'right_ankle',
'LAnkle': 'left_ankle'
 }
target = []
for i in range(markers.shape[0]):
    row = []
    for kp in markers_dict.keys():
        p = np.array([markers[markers_dict[kp]+"_x"][i],markers[markers_dict[kp]+"_y"][i],markers[markers_dict[kp]+"_z"][i]])
        # rotate the 3d point -90 on the x axis (from y up to z up)
        p_n = Rx.dot(p)
        # print(p,p_n)
        row += p_n.tolist()
    target.append(row)
target = np.array(target)

In [ ]:
# Place virtual IMUs on the shoulders
from typing import List, Tuple
right_shoulder: nimble.dynamics.BodyNode = s._nimble.getBodyNode("humerus_r")
translation: np.ndarray = np.array([0.0, 0.0, 0.025])
# z is pointing up, x lateral and y frontal
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)
print(watch_offset)
sensors: List[Tuple[nimble.dynamics.BodyNode, nimble.math.Isometry3]] = [(right_shoulder, watch_offset),]

In [ ]:
# Plot imu position with axes
def plot_imu(imu,axis_length=0.1,imu_key='imu'):
    j_w = imu[0].getWorldTransform().matrix()   # From joint to world
    imu_j = imu[1].matrix()                     # From IMU to joint
    imu_w = j_w @ imu_j # From IMU to world
    
    # Origin
    p0 = imu_w @ np.eye(4)
    gui.nativeAPI().createSphere(key=imu_key+'o', radii=np.array([0.005,0.005,0.005]),pos=p0[0:3,3],color=np.array([0,0,0,1]))
    
    # X coordinate
    px = np.eye(4)
    px[0:3,3] = [axis_length,0,0]
    px = imu_w @ px
    gui.nativeAPI().createLine(key=imu_key+'x',points=[p0[0:3,3],px[0:3,3]],color=np.array([255,0,0,1]))
    
    # Y coordinate
    py = np.eye(4)
    py[0:3,3] = [0,axis_length,0]
    py = imu_w @ py
    gui.nativeAPI().createLine(key=imu_key+'y',points=[p0[0:3,3],py[0:3,3]],color=np.array([0,255,0,1]))
    
    # Z coordinate
    pz = np.eye(4)
    pz[0:3,3] = [0,0,axis_length]
    pz = imu_w @ pz
    gui.nativeAPI().createLine(key=imu_key+'z',points=[p0[0:3,3],pz[0:3,3]],color=np.array([0,0,255,1]))
    
# plot_imu(sensors[0],axis_length=0.05,imu_key='imu'+str(0))

In [ ]:
pos_imu = right_shoulder.getWorldTransform().multiply(watch_offset).matrix()[0:3,3]
s._nimble.setGravity(np.array([0.0, 0.0, -9.81]))
s._nimble.getAccelerometerReadings(sensors)

In [ ]:
# Move the body model using gt to the first position
kps = list(markers_dict.keys())
s.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale()

In [ ]:
# giving the absolute position and orientation of the IMU, get the angle between the floor and the imu Z axis
def get_alpha_angle(imu, plot=True):
    j_w = imu[0].getWorldTransform().matrix()   # From joint to world
    imu_j = imu[1].matrix()                     # From IMU to joint
    imu_w = j_w @ imu_j # From IMU to world
    
    origin = imu_w[:3,3]
    origi_proj = origin.copy()
    origi_proj[-1] = 0 # z = 0
        
    pz = np.eye(4)
    pz[0:3,3] = [0,0,-0.3]
    pz = imu_w @ pz
    
    if plot:
        gui.nativeAPI().createLine(key='origin_projected',points=[origin,origi_proj],color=np.array([0,0,0,0.8]))
        gui.nativeAPI().createLine(key='origin_projected2',points=[origin,pz[:3,3]],color=np.array([0,0,0,0.8]))
    
    # Vectors AB and BC
    u = origi_proj - origin
    v = pz[:3,3] - origin

    # Compute angle in radians
    cos_theta = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    theta_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip to avoid numerical errors

    # Optional: convert to degrees
    theta_deg = np.degrees(theta_rad)
    return theta_deg
    

In [ ]:
def estimate_alpha_angle(a):
    alpha = np.arctan2(np.sqrt(a[0]**2+a[1]**2),a[2])
    return np.degrees(alpha)
    
# estimate_angle(s._nimble.getAccelerometerReadings(sensors))

In [ ]:
# Get ALPHA

# Move right shoulder up
q = s._nimble.getPositions()
q[32] = 0*np.pi/180 # adduction/abduction
q[34] = 0*np.pi/180
q[33] = 20*np.pi/180
s._nimble.setPositions(q)

acc_0 = s._nimble.getAccelerometerReadings(sensors)

# Rendering scene
for i in range(0,-90,-1):
    q[32] = i*np.pi/180
    s._nimble.setPositions(q)
    gui.nativeAPI().renderSkeleton(s._nimble)
    plot_imu(sensors[0],axis_length=0.05,imu_key='imu'+str(0))
    gt_angle = get_alpha_angle(sensors[0],plot=False)
    print(estimate_alpha_angle(s._nimble.getAccelerometerReadings(sensors)),gt_angle)
    time.sleep(0.1)
    print(s._nimble.getAccelerometerReadings(sensors))

In [ ]:
# Move right shoulder down
s.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale()
q = s._nimble.getPositions()
q[32] = 0*np.pi/180 # adduction/abduction
q[34] = 0*np.pi/180
q[33] = 20*np.pi/180
s._nimble.setPositions(q)

acc_0 = s._nimble.getAccelerometerReadings(sensors)

q[32] += 15*np.pi/180
s._nimble.setPositions(q)

for i in range(1000):
    gui.nativeAPI().renderSkeleton(s._nimble)
    q = s._nimble.getPositions()
    acc_target = s._nimble.getAccelerometerReadings(sensors)

    error = acc_0 - acc_target
    loss = np.inner(error, error)


    d_loss_d_acc = 2 * (acc_0 - acc_target)
    d_acc_d_joint_angles = s._nimble.getAccelerometerReadingsJacobianWrt(accs=sensors,wrt=nimble.neural.WRT_POSITION)
    d_loss_d_joint_angles = d_acc_d_joint_angles.T @ d_loss_d_acc

    d_loss_d_joint_angles[0:6] = 0.0 # Do not update the pelvis position in the worlds

    q += 0.00001 * d_loss_d_joint_angles
    s._nimble.setPositions(q)
    # time.sleep(0.1)
    print("loss:",loss,)